In [1]:
import sys
import os
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

import lightgbm as lgb

/Users/tilia/HZ/SmartCleanCityHackZurich19/smartenv/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

In [3]:
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.utils import shuffle

In [4]:
%ls data

180_days_maxT_24hrPrecip.csv
180_days_mean1hrT_1hrPrecip.csv
180_days_meanT_24hrPrecip.csv
2019-09-27-basel-collections.csv
2019-09-27-basel-image-metadata.csv
2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv
2019-09-27-basel-measures-FEAT-TempPrecip.csv
2019-09-27-basel-measures-FEAT.csv
2019-09-27-basel-measures-cleaned.csv
2019-09-27-basel-measures-prediction-cleaned-FEAT.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv
2019-09-27-basel-measures-prediction-cleaned.csv
2019-09-27-basel-measures-prediction.csv
2019-09-27-basel-measures.csv
all_tweets.csv
all_tweets_relcols.csv
distributions/
event_cal.csv
places.txt
twitter.csv
twitter_accounts.txt
twitter_data/
twitter_day_features-TEST.csv
twitter_day_features.csv


## load training data

In [5]:
in_df_train = pd.read_csv("./data/2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv")

in_df_train = in_df_train.drop([c for c in in_df_train.columns if c.startswith('Unnamed')], axis=1)

in_df_train['comb_id'] = in_df_train[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

In [6]:
print(in_df_train.shape)
in_df_train.head()

(58242, 19)


,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,place_name_enc,comb_id
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,11.4,0.0,0,2,163,1175332462-287-2019-04-01 8:40:49
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,11,2,163,25149740-86_9668-2019-04-01 8:40:53
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,8,2,210,117485263-86_11641-2019-04-01 8:40:53
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,11.4,0.0,0,2,163,2621172927-287-2019-04-01 8:40:57
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,11.4,0.0,11,2,65,148833576-86_12436-2019-04-01 8:42:11


#### twitter features

In [7]:
twf_in_train = pd.read_csv('./data/twitter_day_features.csv')
twf_in_train.head()

,day_date,twitter_reldates,tw_place_mentions
0,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN
1,2019-04-02,"2019-04-02, 2019-04-01, 2019-03-31",NaN
2,2019-04-03,"2019-04-03, 2019-04-02, 2019-04-01",NaN
3,2019-04-05,"2019-04-05, 2019-04-04, 2019-04-03",NaN
4,2019-04-09,"2019-04-09, 2019-04-08, 2019-04-07",NaN


In [8]:
import datetime
def get_daydate(date_str):
    '''Returns shortened date string in the format %Y-%m-%d.'''
    
    d = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
       
    return str(datetime.datetime.strftime(d, '%Y-%m-%d'))

in_df_train['day_date'] = in_df_train.date.apply(lambda x: get_daydate(x))

in_df_train.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_german,weekday_german,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,place_name_enc,comb_id,day_date
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,11.4,0.0,0,2,163,1175332462-287-2019-04-01 8:40:49,2019-04-01
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,11,2,163,25149740-86_9668-2019-04-01 8:40:53,2019-04-01
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,8,2,210,117485263-86_11641-2019-04-01 8:40:53,2019-04-01
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,11.4,0.0,0,2,163,2621172927-287-2019-04-01 8:40:57,2019-04-01
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,11.4,0.0,11,2,65,148833576-86_12436-2019-04-01 8:42:11,2019-04-01


In [9]:
in_df_all_train = pd.merge(in_df_train, twf_in_train, how='left', on='day_date')
in_df_all_train.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,...,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,place_name_enc,comb_id,day_date,twitter_reldates,tw_place_mentions
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,...,morning,11.4,0.0,0,2,163,1175332462-287-2019-04-01 8:40:49,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,...,morning,11.4,0.0,11,2,163,25149740-86_9668-2019-04-01 8:40:53,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,...,morning,11.4,0.0,8,2,210,117485263-86_11641-2019-04-01 8:40:53,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,...,morning,11.4,0.0,0,2,163,2621172927-287-2019-04-01 8:40:57,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,...,morning,11.4,0.0,11,2,65,148833576-86_12436-2019-04-01 8:42:11,2019-04-01,"2019-04-01, 2019-03-31, 2019-03-30",NaN


### load test data

In [10]:
%ls ./data/

180_days_maxT_24hrPrecip.csv
180_days_mean1hrT_1hrPrecip.csv
180_days_meanT_24hrPrecip.csv
2019-09-27-basel-collections.csv
2019-09-27-basel-image-metadata.csv
2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv
2019-09-27-basel-measures-FEAT-TempPrecip.csv
2019-09-27-basel-measures-FEAT.csv
2019-09-27-basel-measures-cleaned.csv
2019-09-27-basel-measures-prediction-cleaned-FEAT.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv
2019-09-27-basel-measures-prediction-cleaned.csv
2019-09-27-basel-measures-prediction.csv
2019-09-27-basel-measures.csv
all_tweets.csv
all_tweets_relcols.csv
distributions/
event_cal.csv
places.txt
twitter.csv
twitter_accounts.txt
twitter_data/
twitter_day_features-TEST.csv
twitter_day_features.csv


In [11]:
in_df_test2 = pd.read_csv("./data/2019-09-27-basel-measures-prediction-cleaned-FEAT.csv")

in_df_test2['comb_id'] = in_df_test2[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

in_df_test2 = in_df_test2.drop([c for c in in_df_test2.columns if c.startswith('Unnamed')], axis=1)

in_df_test2 = in_df_test2[['weekday', 'month', 'daytime', 'comb_id']]

print(in_df_test2.shape)

in_df_test2.head()

(7292, 4)


,weekday,month,daytime,comb_id
0,1,6,morning,172611027-86_12812-2019-06-25 6:25:49
1,1,6,morning,3828924874-260-2019-06-25 6:26:22
2,1,6,morning,23913163-86_9255-2019-06-25 6:26:26
3,1,6,morning,23913163-86_17476-2019-06-25 6:26:26
4,1,6,morning,144481239-86_12223-2019-06-25 6:27:21


In [12]:
in_df_test1 = pd.read_csv("./data/2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv")

in_df_test1['comb_id'] = in_df_test1[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

in_df_test1 = in_df_test1.drop([c for c in in_df_test1.columns if c.startswith('Unnamed')], axis=1)

in_df_test1['day_date'] = in_df_test1.date.apply(lambda x: get_daydate(x))

print(in_df_test1.shape)

in_df_test1.head()

(7292, 11)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date
0,172611027,86_12812,2019-06-25 6:25:49,Klingelbergstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25
1,3828924874,260,2019-06-25 6:26:22,Metzerstrasse,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25
2,23913163,86_9255,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25
3,23913163,86_17476,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25
4,144481239,86_12223,2019-06-25 6:27:21,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25


In [13]:
in_df_test1.comb_id.unique().shape

(7292,)

In [14]:
in_df_test = pd.merge(in_df_test1, in_df_test2, how='inner', on='comb_id')

print(in_df_test.shape)

in_df_test.head()

(7292, 14)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date,weekday,month,daytime
0,172611027,86_12812,2019-06-25 6:25:49,Klingelbergstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25,1,6,morning
1,3828924874,260,2019-06-25 6:26:22,Metzerstrasse,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25,1,6,morning
2,23913163,86_9255,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25,1,6,morning
3,23913163,86_17476,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25,1,6,morning
4,144481239,86_12223,2019-06-25 6:27:21,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25,1,6,morning


### twitter features

In [15]:
twf_in_test = pd.read_csv('./data/twitter_day_features-TEST.csv')
twf_in_test.head()

,day_date,twitter_reldates,tw_place_mentions
0,2019-06-25,"2019-06-25, 2019-06-24, 2019-06-23",NaN
1,2019-06-26,"2019-06-26, 2019-06-25, 2019-06-24",theater
2,2019-06-27,"2019-06-27, 2019-06-26, 2019-06-25",theater
3,2019-06-28,"2019-06-28, 2019-06-27, 2019-06-26","st. jakob, theater"
4,2019-08-28,"2019-08-28, 2019-08-27, 2019-08-26",wiesenstrasse


In [16]:
in_df_all_test = pd.merge(in_df_test, twf_in_test, how='left', on='day_date')
print(in_df_all_test.shape)
in_df_all_test.head()

(7292, 16)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date,weekday,month,daytime,twitter_reldates,tw_place_mentions
0,172611027,86_12812,2019-06-25 6:25:49,Klingelbergstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25,1,6,morning,"2019-06-25, 2019-06-24, 2019-06-23",NaN
1,3828924874,260,2019-06-25 6:26:22,Metzerstrasse,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25,1,6,morning,"2019-06-25, 2019-06-24, 2019-06-23",NaN
2,23913163,86_9255,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25,1,6,morning,"2019-06-25, 2019-06-24, 2019-06-23",NaN
3,23913163,86_17476,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25,1,6,morning,"2019-06-25, 2019-06-24, 2019-06-23",NaN
4,144481239,86_12223,2019-06-25 6:27:21,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25,1,6,morning,"2019-06-25, 2019-06-24, 2019-06-23",NaN


In [17]:
def row_feat_dict(row):
    '''
    extract features for one row as a dictionary

    '''
    
    cols = ['place_type', 'weekday', 'month', 'daytime']

    feat_dict = {c:row[c] for c in cols}  # dictionary for one training instance
    
    if str(row['place_name']) == 'nan':
        feat_dict['place_name'] = 'undefined'
    else:
        feat_dict['place_name'] = row.place_name.lower()
    
    
    feat_dict['t_mean_2m_24h:C'] = row['t_mean_2m_24h:C']
    
    feat_dict['precip_24h:mm'] = row['precip_24h:mm']
    
    if str(row['tw_place_mentions']) == 'nan':
        feat_dict['tw_place_mention'] =  'no_tw_places'
    else:
        for tpn in row['tw_place_mentions'].split(', '):
            feat_dict['tw_place_mention'] = tpn
        
    return feat_dict

In [18]:
feats_dict_list_train = in_df_all_train.apply(lambda x: row_feat_dict(x), axis=1)
feats_dict_list_test = in_df_all_test.apply(lambda x: row_feat_dict(x), axis=1)

In [19]:
dict_vec = DictVectorizer()

# fit vectorizer on training data
dict_vec.fit(feats_dict_list_train)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
               sparse=True)

In [20]:
list(dict_vec.get_feature_names())
feature_names = dict_vec.get_feature_names()
feature_names

['daytime=afternoon',
 'daytime=evening',
 'daytime=morning',
 'month',
 'place_name=adlerstrasse',
 'place_name=aeschengraben',
 'place_name=aeschenplatz',
 'place_name=ahornstrasse',
 'place_name=alemannengasse',
 'place_name=allschwilerplatz',
 'place_name=allschwilerstrasse',
 'place_name=amerbachstrasse',
 'place_name=an der tangente',
 'place_name=appenzellerstrasse',
 'place_name=auberg',
 'place_name=augustinergasse',
 'place_name=ausgang schweizerhof',
 'place_name=austrasse',
 'place_name=badischer bahnhof',
 'place_name=bahnhof basel sbb meret-oppenheim-platz',
 'place_name=bahnhof sbb',
 'place_name=barfüsserplatz',
 'place_name=basel, aeschenplatz aagl',
 'place_name=basel, bachgraben',
 'place_name=basel, bahnhof sbb kante k',
 'place_name=basel, belforterstrasse',
 'place_name=basel, bethesda-spital',
 'place_name=basel, blotzheimerstrasse',
 'place_name=basel, emanuel-büchel-strasse',
 'place_name=basel, ensisheimerstrasse',
 'place_name=basel, gellertstrasse',
 'place_

In [21]:
# transform training features to (dense) feature matrix

X_train = dict_vec.transform(feats_dict_list_train)
X_train = X_train.todense()
print(X_train.shape)


(58242, 370)


In [22]:
# transform test features to (dense) feature matrix

X_test = dict_vec.transform(feats_dict_list_test)
X_test = X_test.todense()
print(X_test.shape)

(7292, 370)


In [25]:
y_train = in_df_train.cci.values
y_train

array([3.2       , 3.35764015, 3.35764015, ..., 3.86769316, 4.39927712,
       4.39927712])

In [26]:
random_state = 98
#random_state = 72


pipelines = {
    
    'LinearRegression': LinearRegression(),
    #'DecisionTree': DecisionTreeClassifier(),
    #'GBC': GradientBoostingClassifier(random_state=random_state),
    'LGBMRegressor': lgb.LGBMRegressor(
                                    random_state=random_state,
                                    boosting_type='gbdt', 
                                    learning_rate=0.01,
                                    num_leaves=31,
                                    max_depth=-1,
                                    reg_lambda=0)  # with tuned hyperparams
    
}

In [30]:
#print(sorted(metrics.SCORERS.keys()))

def run_pipelines(X, y, pipelines, num_folds=10, random_state=7):
    '''Runs cross validation (k=num_folds) for all given pipelines'''
    
    regs = dict()
    
    X, y = shuffle(X, y, random_state=random_state)
    
    folds = num_folds

    for reg_name in pipelines:
        print(reg_name)
        reg = pipelines[reg_name]
        
        result = cross_validate(reg, X, y, scoring=['neg_mean_absolute_error'], cv=folds)
        
        print(result)

        reg = reg.fit(X,y)
        
        regs[reg_name] = reg
        
    return regs
        
               

In [31]:
regs = run_pipelines(X_train, y_train, pipelines)

LinearRegression
{'fit_time': array([4.20769477, 3.56459808, 3.67639709, 3.9564178 , 4.91760492,
       3.98484206, 3.42540312, 3.38628078, 3.91004705, 3.32614613]), 'score_time': array([0.00648808, 0.00398612, 0.00392699, 0.00598216, 0.00520515,
       0.00443006, 0.00570703, 0.00412512, 0.00395107, 0.004637  ]), 'test_neg_mean_absolute_error': array([-0.50625276, -0.51549438, -0.51311319, -0.50913027, -0.51565905,
       -0.5125435 , -0.5140271 , -0.50610977, -0.50616421, -0.50527975])}
LGBMRegressor
{'fit_time': array([2.77512908, 2.00652122, 2.64915204, 2.80077386, 1.98378205,
       1.865484  , 2.04708886, 1.87119985, 2.2723    , 2.3809979 ]), 'score_time': array([0.0213089 , 0.04795694, 0.06352472, 0.02493525, 0.02542281,
       0.02793598, 0.02054715, 0.03044105, 0.05152512, 0.02931213]), 'test_neg_mean_absolute_error': array([-0.5200684 , -0.52981512, -0.53351437, -0.52855643, -0.53149852,
       -0.52788916, -0.52602086, -0.52505677, -0.51680937, -0.52296452])}


In [42]:
test_preds_lgbm = regs['LGBMRegressor'].predict(X_test)

test_out_lgbm = in_df_all_test[['osm_id','cci_id','date']].copy()
test_out_lgbm['cci'] = test_preds_lgbm

test_out_lgbm.to_csv('test-PREDS-lgbm.csv', index=None, header=True)

test_out_lgbm.cci.mean()

3.9805775761491424

In [43]:
test_preds_linreg = regs['LinearRegression'].predict(X_test)

test_out_linreg = in_df_all_test[['osm_id','cci_id','date']].copy()
test_out_linreg['cci'] = test_preds_linreg

test_out_linreg.to_csv('test-PREDS-linreg.csv', index=None, header=True)

test_out_linreg.cci.mean()

3.832214996047028